In [1]:
import subprocess
import xarray as xr
import numpy as np
import rasterio

In [2]:
file = 'sample/14112023.nc'
ds = xr.open_dataset(file)

In [3]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 221, latitude: 89, time: 24)
Coordinates:
  * longitude  (longitude) float32 90.0 90.25 90.5 90.75 ... 144.5 144.8 145.0
  * latitude   (latitude) float32 11.0 10.75 10.5 10.25 ... -10.5 -10.75 -11.0
  * time       (time) datetime64[ns] 2023-11-14 ... 2023-11-14T23:00:00
Data variables:
    u          (time, latitude, longitude) float32 ...
    v          (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-02-15 20:51:56 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [8]:
lat = ds['latitude'].values
lon = ds['longitude'].values
u = ds['u'][0].values
v = ds['v'][0].values

In [9]:
ws = np.sqrt(u**2 + v**2)

In [10]:
transform = rasterio.transform.from_bounds(
    west=lon[0],
    south=lat[0],
    east=lon[-1],
    north=lat[-1],
    width=lon.shape[0],
    height=lat.shape[0],
)

In [11]:
# ws = np.flip(ws, axis=0)
with rasterio.open(
    "output/wspd.tif",
    "w",
    driver="GTiff",
    height=lat.shape[0],
    width=lon.shape[0],
    count=1,
    dtype=ws.dtype,
    crs="+proj=latlong",
    transform=transform,
    nodata=-9999,
) as dst:
    dst.write(ws, 1)

In [12]:
subprocess.run(
    [
        "gdaldem",
        "color-relief",
        "-of",
        "JPEG",
        "output/wspd.tif",
        "color/wspd_color.txt",
        "output/wspd.jpg",
    ]
)

CompletedProcess(args=['gdaldem', 'color-relief', '-of', 'JPEG', 'output/wspd.tif', 'color/wspd_color.txt', 'output/wspd.jpg'], returncode=0)